In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from pivottablejs import pivot_ui
import pivottablejs
import qgrid
import matplotlib.pyplot as plt

In [7]:
path='C:\\Users\\camendol\\Desktop\\'
file='PCRs.xlsx'

file_path=path+file

In [8]:
#Open connection to EXCEL
xl_file = pd.ExcelFile(file_path)
xl_file.sheet_names 

['PCRs']

In [9]:
#Read a tab
pcr_data=xl_file.parse('PCRs')

In [10]:
#Apply Date typing
pcr_data['Created']=pd.to_datetime(pcr_data['Creation Date'].str[0:20])
pcr_data['Created_yr_mon'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1))
pcr_data['Created_yr'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1,month=1))
pcr_data['Updated']=pd.to_datetime(pcr_data['Last Update Date'].str[0:20])
pcr_data['Duration']=pcr_data['Updated'] - pcr_data['Created']
pcr_data['Duration']=pcr_data['Duration']/np.timedelta64(1,'D')

pcr_data['QA Expected']=pd.to_datetime(pcr_data['Expected Date to QA'].str[0:20])
pcr_data['QA Delivered']=pd.to_datetime(pcr_data['Actual Date to QA'].str[0:20])
pcr_data['PCR Approved']=pd.to_datetime(pcr_data['New PCR Approval Date'].str[0:20]) 

pcr_data['Update_Elapsed']=(dt.datetime.today() - pcr_data['Updated']).dt.days 
#Maybe bin off day buckets
bin_30s = [-1,15,30,45, 60, 75 ,90, 120, 150, 180, 210, 240, 270, 300, 330, 360, np.inf]
bin_labels = ['15','30', '45','60','75' ,'90', '120', '150','180','210','240','270','300','330','360','361+']
pcr_data['Updated_Seg']=pd.cut(pcr_data['Update_Elapsed'],bin_30s,labels=bin_labels)
pcr_data['Duration_Seg']=pd.cut(pcr_data['Duration'],bin_30s,labels=bin_labels)
#pcr_data.loc[pcr_data['Updated_Seg'].isna()]
#Week Bins
week_bin = [-1,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133,140,147,154,161,168,175,182,np.inf]
week_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27+']
pcr_data['Updated_WKs']=pd.cut(pcr_data['Update_Elapsed'],week_bin,labels=week_labels)

In [11]:
current_team=[ 'Lois Nordstrom'\
              , 'Sue Szubzda'\
              ,'Tricia Huebner'\
              , 'Padmaja Gandhi'\
              , 'Pragada Kumar'\
              , 'Hiranmayee Vemulapalli'\
              , 'Christopher Amendola'\
              ,'Phanikumar Jetti'\
              ,''\
              ,' ']

In [12]:
#Created:Year-to-date
created_YTD=pcr_data.loc[ ((pcr_data['Owner'].isin(current_team) | pcr_data['Owner'].isnull())
                               & (pcr_data['Created']>'2021-01-01'))\
                               ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]\
        .sort_values(by='Created')
len(created_YTD)

59

In [13]:
#Created:Historical
created_hist=pcr_data.loc[ ((pcr_data['Owner'].isin(current_team) | pcr_data['Owner'].isnull())
                               & (pcr_data['Created']>'2019-01-01'))\
                               ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]\
        .sort_values(by='Created')
len(created_hist)

346

In [14]:
completed_21=pcr_data.loc[( (pcr_data['Owner'].isin(current_team))\
                             &  (pcr_data['Impact Kanban State']=='Released')\
                             | (pcr_data['Impact Kanban State']=='UAT'))\
                             & (pcr_data['Created_yr']=='2021-01-01')                            
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]
len(completed_21)

39

In [15]:
active=pcr_data.loc[( (pcr_data['Owner'].isin(current_team))\
                          &((pcr_data['Impact Kanban State']=='Dev') \
                          | (pcr_data['Impact Kanban State']=='Approval')\
                          | (pcr_data['Impact Kanban State']=='Requests')
                          | (pcr_data['Impact Kanban State']=='Code Complete')
                          | (pcr_data['Impact Kanban State']=='Pending Approval')))\
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner'
                                 ,'Blocked')]

In [16]:
len(active.loc[active['Blocked']==True])

11

In [17]:
print('Active Requests 2021: '+str(len(active)))
print('\tCurrently Blocked: '+str(len(active.loc[active['Blocked']==True])))

Active Requests 2021: 31
	Currently Blocked: 11


### Active Requests 2021: 
### January 3rd, 2021  -  34
### February 3rd, 2021 - 51   !!!!!!!!
### February 10th 2021 - 47 !!!!!
### February 26th - 40 !!!!!
### March 3rd - 39 !!!!!
### -----------------------------
### March 20th:
### Active Requests 2021: 36
### Currently Blocked: 12

In [18]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 5]

In [19]:
pivot_ui(created_hist,rows=['Created_yr'],cols=['Created_yr_mon'],rendererName='Bar Chart')

In [20]:
pivot_ui(active,rows=['Updated_WKs'],rendererName='Table')

In [21]:
pivot_ui(completed_21,rows=['Duration_Seg'],rendererName='Table')

In [22]:
#Create Excel sheets for each person
for whom in active['Owner'].unique():
    
    excel_book='C:\\Users\\camendol\\Documents\\PCRs\\'+whom+'.xlsx'
    current=active.loc[(active['Owner']==whom)\
                       ,(  'Name'\
                          ,'Formatted ID'\
                          ,'Owner'\
                          ,'Updated_WKs'\
                          ,'Created_yr'\
                          ,'Duration'\
                          ,'Impact Kanban State')]
    
    comps=completed_21.loc[(completed_21['Owner']==whom),('Name','Formatted ID','Owner','Created_yr_mon','Duration','Impact Kanban State')]

    print(excel_book)
    with pd.ExcelWriter(excel_book) as writer:
        pd.DataFrame(current)\
        .to_excel(writer, sheet_name='Active')
        pd.DataFrame(comps)\
        .to_excel(writer, sheet_name='Completed')

C:\Users\camendol\Documents\PCRs\Padmaja Gandhi.xlsx
C:\Users\camendol\Documents\PCRs\Tricia Huebner.xlsx
C:\Users\camendol\Documents\PCRs\Lois Nordstrom.xlsx
C:\Users\camendol\Documents\PCRs\Hiranmayee Vemulapalli.xlsx
C:\Users\camendol\Documents\PCRs\Pragada Kumar.xlsx
C:\Users\camendol\Documents\PCRs\Sue Szubzda.xlsx
C:\Users\camendol\Documents\PCRs\Christopher Amendola.xlsx


In [23]:
csv_file='C:\\Users\\camendol\\Documents\\PCRs\\Centura_PCRS.csv'

states=['Dev','Pending Approval','Requests']
pcr_data.loc[  ( pcr_data['Name'].str.contains('Centura') 
                & pcr_data['Impact Kanban State'].isin(states))
             ,['Formatted ID','Name','Owner'] ].to_csv(csv_file)

In [24]:
csv_file='C:\\Users\\camendol\\Documents\\PCRs\\Centura_PCRS_total.csv'

states=['Dev','Pending Approval','Requests']
pcr_data.loc[  (( pcr_data['Name'].str.contains('Centura') 
                | pcr_data['Name'].str.contains('Fullwell')) 
               & (pcr_data['Created_yr']>'2019-12-31') )
             ,['Formatted ID','Name','Owner','Created_yr_mon'] ]\
        .sort_values(by='Created_yr_mon')\
        .to_csv(csv_file)

In [25]:
completed_20=pcr_data.loc[( (pcr_data['Owner'].isin(current_team))\
                             &  (pcr_data['Impact Kanban State']=='Released')\
                             | (pcr_data['Impact Kanban State']=='UAT'))\
                             & (pcr_data['Created_yr']=='2020-01-01')                            
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]
len(completed_20)
overall=completed_20.loc[(completed_20['Owner']=='Tricia Huebner'),('Name','Formatted ID','Owner','Created_yr_mon','Duration','Impact Kanban State')]
overall

,Name,Formatted ID,Owner,Created_yr_mon,Duration,Impact Kanban State
63,NYU Fidelis - Member ID suffix,US722304,Tricia Huebner,2020-12-01,35.003472,Released
69,Preferred_One - Add two fields into Input - me...,US720097,Tricia Huebner,2020-12-01,38.030556,Released
80,HealthNet CA: IPro Upload DB Name Update,US717023,Tricia Huebner,2020-12-01,48.946528,Released
90,HealthNet CA PCR - IPRO Upgrade from 8.0 to 11...,US707864,Tricia Huebner,2020-11-01,19.255556,Released
93,Canopy - Clean up request,US704916,Tricia Huebner,2020-10-01,160.982639,Released
94,Canopy Ipro v11.2 migration,US703674,Tricia Huebner,2020-10-01,5.172222,Released
126,H642453 Vera New Source Implementation Tasks: ...,US679277,Tricia Huebner,2020-08-01,236.770139,Released
127,NYU - Healthfirst Non-Par,US679260,Tricia Huebner,2020-08-01,80.775000,Released
133,Vera - new data source (Driscoll),US676434,Tricia Huebner,2020-07-01,5.838889,Released
143,H704847_NYU Null CPT modifiers in L2_II_SERVIC...,US672728,Tricia Huebner,2020-07-01,102.553472,Released


In [26]:
pivot_ui(created_hist,rows=['Created_yr'],cols=['Created_yr_mon'],rendererName='Bar Chart',outfile_path="C:\\Users\\camendol\\Desktop\\pivot.html")